In [3]:
%matplotlib notebook

In [1]:
import numpy as np
import random
import itertools
from graph_tool import Graph, GraphView
from graph_tool.draw import graph_draw
from tqdm import tqdm

from matplotlib import pyplot as plt

from viz_helpers import lattice_node_pos
from minimum_steiner_tree import min_steiner_tree
from cascade_generator import si, observe_cascade
from eval_helpers import infection_precision_recall
from graph_helpers import remove_filters, load_graph_by_name, gen_random_spanning_tree

from inference import infer_infected_nodes
from experiment import gen_input

In [2]:
graph_name = 'karate'
g = load_graph_by_name(graph_name)

In [16]:
n_rounds = 100
n_samples = 200
subset_size = 50
stop_fraction = 0.25
obs_fraction = 0.5

In [19]:
sampling_scores = []
sp_trees = [gen_random_spanning_tree(g) for _ in range(n_samples)]
for i in tqdm(range(n_rounds)):
    obs, c = gen_input(g, stop_fraction=stop_fraction, q=obs_fraction)
    preds = infer_infected_nodes(g, obs, method='sampling', sp_trees=sp_trees)
    prec, rec = infection_precision_recall(set(preds), c, obs)
    sampling_scores.append((prec, rec))


 14%|█▍        | 28/200 [00:10<01:00,  2.83it/s]Exception in thread Thread-13:
Traceback (most recent call last):
  File "/usr/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.5/dist-packages/tqdm/_tqdm.py", line 102, in run
    for instance in self.tqdm_cls._instances:
  File "/usr/lib/python3.5/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration
100%|██████████| 200/200 [01:10<00:00,  2.96it/s]


In [20]:
subset_sampling_scores = []
for i in tqdm(range(n_rounds)):
    obs, c = gen_input(g, stop_fraction=stop_fraction, q=obs_fraction)
    preds = infer_infected_nodes(g, obs, method='sampling', sp_trees=sp_trees, subset_size=subset_size)
    prec, rec = infection_precision_recall(set(preds), c, obs)
    subset_sampling_scores.append((prec, rec))

100%|██████████| 200/200 [01:07<00:00,  2.92it/s]


In [42]:
mst_scores = []
for i in tqdm(range(n_rounds)):
    obs, c = gen_input(g, stop_fraction=stop_fraction, q=obs_fraction)
    preds = infer_infected_nodes(g, obs, method='min_steiner_tree')
    prec, rec = infection_precision_recall(set(preds), c, obs)
    mst_scores.append((prec, rec))

100%|██████████| 100/100 [00:02<00:00, 38.02it/s]


In [26]:
import pandas as pd
sampling_scores = np.asarray(sampling_scores)
df = pd.DataFrame(sampling_scores, columns=['prec', 'rec'])
df.describe()

,prec,rec
count,200.000000,200.000000
mean,0.592583,0.252190
std,0.358632,0.175954
min,0.000000,0.000000
25%,0.333333,0.166667
50%,0.666667,0.200000
75%,1.000000,0.400000
max,1.000000,0.800000


In [27]:
subset_sampling_scores = np.asarray(subset_sampling_scores)
df = pd.DataFrame(subset_sampling_scores, columns=['prec', 'rec'])
df.describe()

,prec,rec
count,200.000000,200.000000
mean,0.623250,0.206565
std,0.414650,0.160966
min,0.000000,0.000000
25%,0.250000,0.107143
50%,0.666667,0.200000
75%,1.000000,0.333333
max,1.000000,0.800000


In [44]:
mst_scores = np.asarray(mst_scores)
mst_scores.mean(axis=0)

array([ 0.09299603,  0.10905952])